## Machine Learning

In [1]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses

    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return [('gini', gini_score)]

In [11]:
# Set xgboost parameters
params = {}
params['objective'] = 'binary:logistic'
params['eta'] = [0.02,0.03,0.04]
params['silent'] = True
params['max_depth'] = range(2,8)
params['subsample'] = 0.9
params['colsample_bytree'] = 0.9
params['eval_metric'] = 'auc'
params['silent'] = True

In [15]:
# Train the model! We pass in a max of 10,000 rounds (with early stopping after 100)
# and the custom metric (maximize=True tells xgb that higher metric is better)

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
import gc


print('loading files...')
train = pd.read_csv('../../data/train.csv', na_values=-1) 
test = pd.read_csv('../../data/test.csv', na_values=-1)
col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(col_to_drop, axis=1)  
test = test.drop(col_to_drop, axis=1)  

for c in train.select_dtypes(include=['float64']).columns:
    train[c]=train[c].astype(np.float32)
    test[c]=test[c].astype(np.float32)
for c in train.select_dtypes(include=['int64']).columns[2:]:
    train[c]=train[c].astype(np.int8)
    test[c]=test[c].astype(np.int8)    

print(train.shape, test.shape)

loading files...
(595212, 39) (892816, 38)


In [18]:

    
# xgb
#'eta': 0.02, 'max_depth': 4
params = {'eta': 0.03, 'max_depth': 5, 'subsample': 0.9, 'colsample_bytree': 0.9, 
          'objective': 'binary:logistic', 'eval_metric': 'auc', 'silent': True,'maximize':True}

X = train.drop(['id', 'target'], axis=1)
features = X.columns
X = X.values
y = train['target'].values
sub=test['id'].to_frame()
sub['target']=0

nrounds=2000  # need to change to 2000
kfold = 9  # need to change to 5
skf = StratifiedKFold(n_splits=kfold, random_state=0)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    d_train = xgb.DMatrix(X_train, y_train) 
    d_valid = xgb.DMatrix(X_valid, y_valid) 
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    
    xgb_model = xgb.train(params, d_train, nrounds, watchlist, early_stopping_rounds=100, 
                          feval=gini_xgb, maximize=True, verbose_eval=100)
    
    sub['target'] += xgb_model.predict(xgb.DMatrix(test[features].values), 
                        ntree_limit=xgb_model.best_ntree_limit+50) / (2*kfold)
    
sub.to_csv('submission.csv', index=False, float_format='%.5f')
gc.collect()
sub.head(2)


 xgb kfold: 1  of  9 : 
[0]	train-gini:0.21324	valid-gini:0.229552
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-gini:0.275535	valid-gini:0.279866
[200]	train-gini:0.318554	valid-gini:0.291786
[300]	train-gini:0.344297	valid-gini:0.295325
[400]	train-gini:0.36365	valid-gini:0.296005
[500]	train-gini:0.379692	valid-gini:0.295346
Stopping. Best iteration:
[411]	train-gini:0.36536	valid-gini:0.296187

 xgb kfold: 2  of  9 : 
[0]	train-gini:0.213905	valid-gini:0.205672
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-gini:0.277686	valid-gini:0.239784
[200]	train-gini:0.320679	valid-gini:0.263814
[300]	train-gini:0.346928	valid-gini:0.273024
[400]	train-gini:0.366211	valid-gini:0.276095
[500]	train-gini:0.380649	valid-gini:0.276261
Stopping. Best iteration:
[478]	train-gini

,id,target
0,0,0.013114
1,1,0.013764


In [19]:
xgb_model.best_score #before 0.276938

0.288663